In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image, make_grid
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from diffusers import UNet2DModel, DDPMScheduler, DDIMScheduler

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 超参数配置
class Config:
    # 扩散过程
    timesteps = 1000
    image_size = 128  # 图像大小
    channels = 3      # 图像通道数

    # 网络结构
    n_feat = 128      # 基础特征数
    n_cfeat = 5       # 上下文特征维度

    # 训练参数
    batch_size = 16
    n_epoch = 200     # 增加训练轮数
    lrate = 2e-4      # 调整学习率

    # 路径配置
    data_path = "/content/drive/MyDrive/Beesimages"
    save_dir = "/content/drive/MyDrive/generatedimages"
    os.makedirs(save_dir, exist_ok=True)

cfg = Config()

# 数据增强
transform = transforms.Compose([
    transforms.Resize(cfg.image_size),
    transforms.CenterCrop(cfg.image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 数据加载
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = [os.path.join(root_dir, f)
                          for f in os.listdir(root_dir)
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, torch.zeros(cfg.n_cfeat)

# 增强版模型
class EnhancedDiffusionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.unet = UNet2DModel(
            sample_size=cfg.image_size,
            in_channels=cfg.channels,
            out_channels=cfg.channels,
            layers_per_block=3,
            block_out_channels=(cfg.n_feat, cfg.n_feat*2, cfg.n_feat*4, cfg.n_feat*8),
            attention_head_dim=8,
            down_block_types=(
                "DownBlock2D",
                "AttnDownBlock2D",
                "AttnDownBlock2D",
                "AttnDownBlock2D"),
            up_block_types=(
                "AttnUpBlock2D",
                "AttnUpBlock2D",
                "UpBlock2D",
                "UpBlock2D")
        )
        # 训练用噪声调度器
        self.train_scheduler = DDPMScheduler(
            num_train_timesteps=cfg.timesteps,
            beta_schedule="squaredcos_cap_v2",
            prediction_type="epsilon"
        )
        # 推理用调度器(加速)
        self.infer_scheduler = DDIMScheduler(
            num_train_timesteps=cfg.timesteps,
            beta_schedule="squaredcos_cap_v2",
            prediction_type="epsilon"
        )

    def forward(self, x, t, c=None):
        return self.unet(x, t).sample

# 训练过程
def train():
    model = EnhancedDiffusionModel().to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lrate, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg.n_epoch)

    dataset = CustomDataset(cfg.data_path, transform)
    dataloader = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True, num_workers=4)

    for epoch in range(cfg.n_epoch):
        model.train()
        pbar = tqdm(dataloader, desc=f"Epoch {epoch}/{cfg.n_epoch}")
        loss_ema = None

        for x, c in pbar:
            x = x.to(device)
            c = c.to(device)

            t = torch.randint(0, cfg.timesteps, (x.size(0),)).to(device)
            noise = torch.randn_like(x)
            noisy_x = model.train_scheduler.add_noise(x, noise, t)

            pred_noise = model(noisy_x, t, c)
            loss = F.mse_loss(pred_noise, noise)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            loss_ema = loss.item() if loss_ema is None else 0.9*loss_ema + 0.1*loss.item()
            pbar.set_postfix(loss=f"{loss_ema:.4f}")

        scheduler.step()

        if epoch % 50 == 0 or epoch == cfg.n_epoch-1:
            state = {
                'epoch': epoch,
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'loss': loss_ema,
            }
            torch.save(state, f"{cfg.save_dir}checkpoint_{epoch}.pth")
            generate_samples(model, epoch)

# 生成样本
def generate_samples(model, epoch, n_samples=9):
    model.eval()
    with torch.no_grad():
        samples = torch.randn(n_samples, cfg.channels, cfg.image_size, cfg.image_size).to(device)
        model.infer_scheduler.set_timesteps(50)  # 加速采样

        for t in tqdm(model.infer_scheduler.timesteps, desc="Sampling"):
            t_batch = torch.full((n_samples,), t, device=device)
            noise_pred = model(samples, t_batch, None)
            samples = model.infer_scheduler.step(noise_pred, t, samples).prev_sample

        samples = (samples.clamp(-1, 1) + 1) / 2
        grid = make_grid(samples, nrow=3)

        plt.figure(figsize=(10, 10))
        plt.imshow(grid.permute(1, 2, 0).cpu())
        plt.axis('off')
        plt.savefig(f"{cfg.save_dir}samples_{epoch}.png", bbox_inches='tight')
        plt.close()

if __name__ == "__main__":
    train()